In [2]:
%load_ext Cython

In [9]:
%%cython -a --cplus
from libc.stdint cimport uint64_t
from libcpp.string cimport string
cdef string hello="Hello World!"
print hello

CompileError: command 'C:\\Program Files (x86)\\Microsoft Visual Studio 9.0\\VC\\BIN\\amd64\\cl.exe' failed with exit status 2

In [3]:
%%cython -a --cplus

# include "c4.pxi"

#cython: cdivision = True
#cython: embedsignature = True
import sys
# from libc.stdint cimport uint64_t
import random
ctypedef unsigned long long int64
ctypedef int64 int128
ctypedef int128 move
ctypedef int128 bitboard

from numpy.math cimport INFINITY
cdef double INF=INFINITY
    
'''
bitboards for the board represented by which bit (example is 6x7 board)
0  0  0  0  0  0  0
5 12 19 26 33 40 47
4 11 18 25 32 39 46
3 10 17 24 31 38 45
2  9 16 23 30 37 44
1  8 15 22 29 36 43
0  7 14 21 28 35 42
'''

cdef enum:
    MAX_ROWS = 16
    MAX_COLUMNS = 16
    

cdef int ROWS = 6
cdef int COLUMNS = 7
cdef int COL_MASK = (1 << (ROWS+1))-1
cdef int *DeBruijn = [0,1,28,2,29,14,24,3,30,22,20,15,25,17,4,8,31,27,13,23,21,19,16,7,26,12,18,6,11,5,10,9]
random.seed(42)
cdef unsigned long long hash_table[2][128]
hash_table = get_zob()

cdef int population(bitboard bb):
    #counts the number of on bits in the bitboard
    cdef int pop = 0    
    while bb:
        pop += 1
        bb &= (bb -1)
    return pop

cdef unsigned int bitindex(move bitmove): #DeBruijn method by multiply and lookup
    return DeBruijn[<unsigned long> (bitmove * <move> 0x077CB531) >> 27]

cdef int log(bitboard v): #binary method for finding log
    cdef int64 b[6]
    cdef int64 S[6]
    b = [0x2, 0xC, 0xF0, 0xFF00, 0xFFFF0000, 0xFFFFFFFF00000000]
    S = [1, 2, 4, 8, 16, 32]
    cdef int i=5
    cdef int r = 0
    while i >= 0:   
        if v & b[i]:
            v >>= S[i]
            r |= S[i]   
        i -= 1
    return r
    

cdef void insert(int* scores, move* moves, int current, move bitmove, int size):
    cdef int i,j
    i = 0
    while current <= scores[i] and i < size: #find the index to insert
        i += 1
    while size > i: 
        scores[size] = scores[size-1]
        moves[size]  = moves[size-1]
        size -= 1
    scores[i] = current
    moves[i] = bitmove

cdef get_zob():
    cdef:
        unsigned long long zob[2][128]
        int i,j
    for i in range(2):
        for j in range(128):
            zob[i][j] = random.getrandbits(64)
    return zob

# cdef class Score(object):
#     cdef:
#         public double result
#         public int depth
        
#     def __cinit__(self,double result=0,int depth=0):
#         self.result = result
#         self.depth = depth
#     def __lt__(self,other):
#         if self.result != other.result:
#             return self.result < other.result
#         else:
#             return self.depth * self.result > other.depth * self.result
#     def __gt__(self,other):
#         return other < self
#     def __le__(self,other):
#         if self.result != other.result:
#             return self.result < other.result
#         else:
#             return self.depth * self.result >= other.depth * self.result
#     def __ge__(self,other):
#         return other <= self
#     def __eq__(self,other):
#         return self.result==other.result and self.depth==other.depth
#     def __ne__(self,other):
#         return not self==other
#     def __neg__(self):
#         self.result *= -1
    

cdef class Board(object):
    
    cdef public:
        int player,value,rows,columns,over,result
        object log
        int height[64]
        bitboard boards[2],bitlog[MAX_COLUMNS * MAX_ROWS]
        bitboard up,up2,up3,down,down2,down3,vert,vert2,vert3,hor,hor2,hor3,MASK,BOTTOM
        bitboard key
        int64 nodes_visited
        int clog[MAX_COLUMNS * MAX_ROWS]
        int ilog
        int strong

        
        
    def __cinit__(self,int rows = 6,int columns = 7):
        global ROWS
        global COLUMNS
        global COL_MASK
        ROWS = rows
        COLUMNS = columns
        if COLUMNS*(ROWS+1)> 64:
            sys.exit('dimensions too large for 64-bit boards')
        COL_MASK = (<bitboard>1 << <bitboard> (ROWS+1))-1
        cdef move i,bitmove
        self.player = 0
        self.value = 1
        self.log = []
        self.over = 0
        self.result = 0
        self.rows = rows
        self.columns = columns
        self.key = 0
        self.ilog = 0
        self.nodes_visited = 0
        
        self.boards = [0,0]
        self.up = ROWS + 2
        self.up2 = self.up*2 #seeing if only doing this calculation once speeds things up
        self.up3 = self.up*3
        
        self.down = ROWS
        self.down2 = self.down*2
        self.down3 = self.down*3
        
        self.vert = 1
        self.vert2 = self.vert*2
        self.vert3 = self.vert*3
        
        self.hor = ROWS + 1
        self.hor2 = self.hor*2
        self.hor3 = self.hor*3
    
        self.BOTTOM = 0
        self.MASK = 0
            
        for i in xrange(COLUMNS):
            self.BOTTOM |= <bitboard> 1 << i * <bitboard>(ROWS + 1)
        for i in xrange(ROWS):
            self.MASK |= self.BOTTOM << i
            

            
        
        
# keeps track of how many pieces are in each column

    cdef int check_win(self, int player):
        cdef bitboard bb = self.boards[player]
        if     (bb & (bb>>self.vert) & (bb>>self.vert2) & (bb>>self.vert3) |
                bb & (bb>>self.hor)  & (bb>>self.hor2)  & (bb>>self.hor3)  |
                bb & (bb>>self.up)   & (bb>>self.up2)   & (bb>>self.up3)   |
                bb & (bb>>self.down) & (bb>>self.down2) & (bb>>self.down3)):
            return 1
        return 0
    
    cpdef is_key(self,key):
        return self.get_key() == key
    
    cdef bitboard get_key(self):
        return self.boards[0] | ((self.boards[0]|self.boards[1])+self.BOTTOM)
    
    cdef bitboard get_symm_keys(self,bitboard* keys):        
        cdef bitboard COL_MASK = (<bitboard> 1 << (ROWS+<bitboard>1))-<bitboard>1
        cdef int i
        keys[0] = self.get_key()   
        keys[1] = 0
        for i in xrange(COLUMNS):
            keys[1] ^= (COL_MASK & (keys[0] >> ((ROWS+<bitboard>1) * <bitboard>i))) << (
                (ROWS+<bitboard>1)*(COLUMNS-<bitboard>(i+1)))
        return 1 + (keys[0] != keys[1])
    
    cpdef get_key_wrapper(self):
        return self.get_key()        
    cdef int check_full(self):
        return self.boards[0]|self.boards[1] == self.MASK
    
    cdef int check_over(self):
        return self.check_win(0) or self.check_win(1) or self.check_full()   
    
    cdef double get_score(self):
        if self.check_win(0):
            return self.rows*self.columns - self.ilog + 1
        if self.check_win(1):
            return -(self.rows*self.columns - self.ilog + 1)
        return 0   
    
    cdef double max_score(self,int n): #winning next move  
        #the int n is just for communicating with move generator. There is probably a more transparent way of doing this
        return max(0,self.rows*self.columns - self.ilog + 1 - n)
        
    cdef double min_score(self,int n): #losing in n plies
        return min(0,-(self.rows*self.columns - self.ilog + 1 - n))
        
    cdef int next_turn_result(self):
        cdef bitboard bitmap = self.get_legal_bitmap()
        if bitmap == -1:
            return 1
        if bitmap == 0:
            return -1
        return 0
        
        
    cdef int move_score(self,move bitmove, int player):
        
        # A simple heuristic for move ordering
        # return the number of winning moves on the board after moving to move, minus the losing moves.
        
        cdef bitboard p1,p2,wins,legal,legalwins
        
        if player == 0:
            p1 = self.boards[0] ^ bitmove
            p2 = self.boards[1]
        else:
            p1 = self.boards[0]
            p2 = self.boards[1] ^ bitmove
            
        #check if there is a 2-way threat
        wins = self.winning_plays(p1,p2,player)
        legal = (p1|p2) + self.BOTTOM
        legalwins = wins & legal 
        # check if board has unblockable win (2-way or stacked threats)
        if ((legalwins) & (legalwins-1)) or (legalwins & (wins >> <bitboard> 1)): 
            return 128 #signal a win
        else:
            return population(wins)
        
    cdef bitboard winning_plays(self,bitboard p1,bitboard p2,int player):
        #bitboard argument for the case that you want to check boards other than the current board state
        #13 patterns to check for 4 each for horizonal, ldiagonal and rdiagonal, as well as one vertical
        #going to check which places win the game, then remove moves that have already been played with self.MASK
        cdef bitboard holder, winners,bb
        if player == 0:
            bb = p1
        else:
            bb = p2
        winners = 0
        
        #vertical
        winners |= (bb << self.vert) & (bb << self.vert2) & (bb << self.vert3)
        
        #horizontal
        #can do two patterns at a time with an intermitant bitboard
        holder = (bb << self.hor) & (bb << self.hor2) # xx_
        winners |= holder & (bb << self.hor3) #xxx_
        winners |= holder & (bb >> self.hor) # xx_x
        holder = (bb >> self.hor) & (bb >> self.hor2) # _xx
        winners |= holder & (bb >> self.hor3) #_xxx
        winners |= holder & (bb << self.hor) # x_xx
        
        #similarly for the diagonals
        holder = (bb << self.up) & (bb << self.up2) # xx_
        winners |= holder & (bb << self.up3) #xxx_
        winners |= holder & (bb >> self.up) # xx_x
        holder = (bb >> self.up) & (bb >> self.up2) # _xx
        winners |= holder & (bb >> self.up3) #_xxx
        winners |= holder & (bb << self.up) # x_xx
        
        holder = (bb << self.down) & (bb << self.down2) # xx_
        winners |= holder & (bb << self.down3) #xxx_
        winners |= holder & (bb >> self.down) # xx_x
        holder = (bb >> self.down) & (bb >> self.down2) # _xx
        winners |= holder & (bb >> self.down3) #_xxx
        winners |= holder & (bb << self.down) # x_xx

        winners &= (self.MASK^(p1|p2)) #only empty, legal squares are wanted
        
        return winners
    
    cdef bitboard get_legal_bitmap(self):
        cdef bitboard legal , winning , bitmove , bb , viable , blocking
        cdef int col , i, num_moves,value
        cdef int scores[MAX_COLUMNS]
        
        legal = ((self.boards[0]|self.boards[1]) + self.BOTTOM) & self.MASK
        winning = self.winning_plays(self.boards[0],self.boards[1],self.player)
        bb = legal&winning #is there a winning move 
        if bb:
            return -1

        blocking = self.winning_plays(self.boards[0],self.boards[1],self.player^1) #opponent's winning moves
        bb = legal&blocking #do we need to block a win?

        if bb:
            if bb & (blocking >> <bitboard> 1) or (bb & (bb -1)): 
# bb&(bb-1) removes the least significant bit, which evaluates to true only when there is more than one bit set
# if there are two stacked wins or two playable wins
                return 0
            return bb
        viable = legal & ~(blocking >> <bitboard> 1) #don't consider moves which let the opponent win
        return viable
        
        
    cdef int get_legal_smart(self,move* moves):
        #return the number of moves sent back
        #flaged -1 if win 0 if loss
        #assumes that can't lose on next move
        cdef bitboard legal , winning , bitmove , bb , viable , blocking
        cdef int col , i, num_moves,value
        cdef int scores[MAX_COLUMNS]
        
        legal = ((self.boards[0]|self.boards[1]) + self.BOTTOM) & self.MASK
        blocking = self.winning_plays(self.boards[0],self.boards[1],self.player^1) #opponent's winning moves
        bb = legal&blocking #do we need to block a win?
        if bb:
            moves[0] = bb
            return 1                
        viable = legal & ~(blocking >> <bitboard> 1) #don't consider moves which let the opponent win
        if not viable: #every move loses, return any move
            moves[0] = legal & (~legal + 1)
            return 0
        num_moves = 0
        while viable:
            bitmove = viable & (~viable + 1) #lowest set bit            
            value = self.move_score(bitmove,self.player) # want some separation to add weight to centre
            if value == 128: #sentinal for two way win
                moves[0] = bitmove
                return -3
            col = log(bitmove) / (ROWS+1)
            value *= COLUMNS
            value += min(col , COLUMNS - 1 - col)
            insert(scores,moves,value,bitmove,num_moves)            
            num_moves += 1
            viable ^= bitmove
        return num_moves

    cdef int get_legal_fast(self,move* moves):
        cdef bitboard legal
        cdef move bitmove
        cdef int n
        n=0
        legal = ((self.boards[0]|self.boards[1]) + self.BOTTOM) & self.MASK
        while legal:
            bitmove = legal & (~legal + 1)
            moves[n] = bitmove
            legal ^= bitmove
            n+=1
        return n
    
    cdef int get_legal(self,move* moves):
        return self.get_legal_smart(moves)
            
    cpdef object p_get_legal(self):
        cdef int i
        cdef object l
        l = [COLUMNS/2 + (1 - 2*(i%2))*((i+1)/2) for i in xrange(COLUMNS)] #starting from middle
        return [move for move in l if self.height[move] < ROWS]
    
    cpdef p_update(self,int m):
        self.boards[self.player] ^= (<bitboard> 1) << (m * (1+self.rows) + self.height[m])
        self.key ^= hash_table[self.player][m * (1+self.rows) + self.height[m]]
        self.player ^= 1
        self.height[m] += 1
        self.log.append(m)
        self.clog[self.ilog] = m
        self.bitlog[self.ilog] = (<bitboard> 1) << (m * (1+self.rows) + self.height[m])
        self.ilog += 1
        self.nodes_visited += 1
        
    cdef update(self,move bitmove):
#         assert(not bitmove & (bitmove - 1))
        self.boards[self.player] ^= bitmove
        self.player ^= 1
        self.bitlog[self.ilog] = bitmove
        self.ilog += 1
        self.nodes_visited += 1
        
    cpdef p_erase(self):
        cdef int m
        m = self.log.pop()
        self.ilog -= 1
        self.height[m] -= 1
        self.player ^= 1
        self.key ^= hash_table[self.player][m * (1+self.rows) + self.height[m]]
        self.boards[self.player] ^= (<bitboard> 1) << (m * (1+self.rows) + self.height[m])
        
    cdef erase(self):
        cdef move bitmove
        bitmove = self.bitlog[self.ilog - 1]
        self.ilog -= 1
        self.player ^= 1
        self.boards[self.player] ^= bitmove
        
    def get_nodes_visited(self):
        return self.nodes_visited
        
    def get_boards(self):
        return self.boards
    
    def get_player(self):
        return self.player
    
    def get_log(self):
        return self.log
    
    def get_dimensions(self):
        return self.rows,self.columns
    
    def print_vitals(self):
        print 'boards',self.boards
        print "player's turn" , self.player
        print "column heights", [self.height[i] for i in xrange(self.columns)]
        print "Is the game over?", self.is_over()
        print "who won?", self.result
        print "log", self.log
        print "nodes visited", self.nodes_visited
        print "key", self.key
        print "MASK", self.MASK
        print "ROWS", ROWS
        print "COLUMNS",COLUMNS
    

    cpdef print_board(self):
        cdef int i,j
        big = ''
        for i in xrange(self.rows):
            s = '|'
            for j in xrange(self.columns):
                s += ' '
                if (self.boards[0] >> i + j * (1 + self.rows)) & 1:
                    s += 'x'
                elif (self.boards[1] >> i + j * (1 + self.rows)) & 1:
                    s += 'o'
                else:
                    s += ' '
                s += '|'
            big = s + '\n' + big
        print big
            

    cpdef is_over(self):
        return self.check_over()
    cpdef get_result(self):
        return self.result
    
    
cpdef char_debug():
    cdef char a,b
    a = <char>'n'
    print a
    print a == <char>'n'
    b=<char>'l'
    print b
    print a == b
    print a == <char>'n'





    
# ###################################################################
    
# include "ab.pxi"

# ###################################################################

cdef enum:
#     PRIME = 15000017
#     PRIME = 11111119    
    PRIME = 1000003
    
cdef int64 pushes = 0
cdef int64 bigpulls = 0
cdef int64 newpulls = 0
#table will have (key, value, depth, best move, type of node, size of node)
cdef struct position_data:
    bitboard key
    double value
    int depth
    move move
    int64 size
    char node
    
cdef struct twobigentry:
    position_data small
    position_data big    

cdef twobigentry twobig[PRIME]

cdef position_data init_position_data():
    cdef position_data data
    data.key = -1
    data.value = -1
    data.node  = 'n'
    data.depth = -1
    data.move = -1
    data.size = 0
    return data
    
cdef void init_twobig():
    for i in xrange(PRIME):
        twobig[i].small = init_position_data()
        twobig[i].big = init_position_data()
        
cdef push_twobig(int64 key,double score,int128 move,int depth,int128 size,char node):
    cdef int64 index
    cdef int64 temp
    global pushes
    index = key % <int64> PRIME
    pushes = pushes+1

    if key == twobig[index].big.key or size > twobig[index].big.size:
        twobig[index].big.value = score
        twobig[index].big.depth = depth
        twobig[index].big.node = node
        if size > twobig[index].big.size:
            twobig[index].big.size = size
        twobig[index].big.move = move
        twobig[index].big.key = key
        
    else:
        twobig[index].small.value = score
        twobig[index].small.depth = depth
        twobig[index].small.node  = node
        if key != twobig[index].small.key or size > twobig[index].small.size:
            twobig[index].small.size = size
        twobig[index].small.key = key
        twobig[index].small.move = move
            
cdef position_data default
default = init_position_data()
    
#@cython.cdivision(True)
cdef position_data* pull_twobig(int64 key):
    cdef int64 index = key % <int64> PRIME
    if key == twobig[index].big.key:
        return &twobig[index].big
    elif key == twobig[index].small.key:
        return &twobig[index].small
    return &default


cdef int symply = 22
cpdef change_symply(int n):
    global symply
    symply = n

cdef double ab(Board board,int depth, double alpha, double beta,dict book,int ply , int* nodes_visited):
    nodes_visited[0] += 1
    
    if depth == 0:
        return 0
    cdef bitboard thiskey
    thiskey = board.get_key()
    if ply >= 0 and thiskey in book: 
        return book[thiskey]
    

    cdef move moves[MAX_COLUMNS]
    cdef int i,j,t
    cdef int num_legal
    cdef object legal_moves
    cdef int64 rand
    cdef position_data* entry
    cdef int64 size
    cdef int player 
    cdef double best = -INF
    cdef double current = -1
    cdef move candidate = -1
    cdef int passed_score[1]
    cdef move passed_move[1]
    cdef int scores[MAX_COLUMNS]
    cdef double original_alpha = alpha
    cdef double original_beta = beta
    cdef char node = 'n'
    cdef int counter = 0
    cdef int table_move=0
    cdef bitboard keys[MAX_COLUMNS]
    cdef position_data* thisentry
    cdef int n_keys
    cdef int next_turn
        
    
    passed_move[0] = 0
    passed_score[0] = -1
    global pushes
    global d
    global symply
    
    size = nodes_visited[0] #store current visit count to compare to later
    
    # window narrowing
    alpha = max(alpha,board.min_score(2))
    beta = min(beta,board.max_score(1))
    if alpha>=beta:
        return alpha
    
    if board.player == 0:
        player = 1
    else:
        player = -1
    if board.ilog <symply:
        n_keys = board.get_symm_keys(keys)
    else:
        n_keys = 1
        keys[0] = thiskey
    thisentry = pull_twobig(thiskey)

    for i in xrange(n_keys):
        entry = pull_twobig(keys[i])
        if entry[0].node != <char>'n':
            if entry[0].node == 'x':
                return entry[0].value

            if entry[0].depth >= depth:

                if entry[0].node == <char>'e': # if the value is certain, we use it
                    return entry[0].value


                if entry[0].node == <char>'u' and beta > entry[0].value:
                    beta = entry[0].value


                if entry[0].node == <char>'l' and alpha< entry[0].value:
                    alpha = entry[0].value
    
            
# # the move from the transposition talbe should be tried first, since it was the best move earlier in the search
# # This should really just be moved to the first entry in the legal moves list, 
# # but since it will be in the transposition table when it comes up, it doesn't use much time
                   

    best = alpha
    candidate = thisentry[0].move

    if alpha < beta:
            #check if there is a quick win or loss early
        next_turn = board.next_turn_result()
        if next_turn == 1:
            node = <char> 'x'
            size = nodes_visited[0] - size
    #         assert(not moves[0] & (moves[0]-1)),'forced win'
            push_twobig(board.get_key(),board.max_score(1),-1,depth,size+1,node)
            return board.max_score(1)
        if next_turn == -1:
            node = <char> 'x'
            size = nodes_visited[0] - size
    #         assert(not moves[0] & (moves[0]-1)),'inevitable loss'
            push_twobig(board.get_key(),board.min_score(2),-1,depth,size+1,node)
            return board.min_score(2)
        # window narrowing (again)
        alpha = max(alpha,board.min_score(4))
        beta = min(beta,board.max_score(3))
        best = alpha
        
    if alpha < beta:

        num_legal = board.get_legal(moves)
#         num_legal = board.get_legal_fast(moves)

#if the move generator is able to filter bad moves

        if num_legal < 0: #sentinal for forced win in -num_legal plies
            node = <char> 'x'
            size = nodes_visited[0] - size
            assert(not moves[0] & (moves[0]-1)),'forced win'
            push_twobig(board.get_key(),board.max_score(-num_legal),moves[0],depth,size+1,node)
            return board.max_score(-num_legal)
        if num_legal == 0: #sentinal for loss next move
            node = <char> 'x'
            size = nodes_visited[0] - size
            assert(not moves[0] & (moves[0]-1)),'inevitable loss'
            push_twobig(board.get_key(),board.min_score(2),moves[0],depth,size+1,node)
            return board.min_score(2)        
    
        if candidate == -1:
            candidate = moves[0]
        
        if thisentry[0].node == 'l' or thisentry[0].node == 'e': #try best move from earlier attempt first
            table_move = 1 #flag that we tried the table move
            counter += 1
#             assert(thisentry.move in [moves[i] for i in xrange(num_legal)])
            board.update(thisentry[0].move)
            if board.check_over():
                current = board.get_score()*(1-2*board.player)
            else:
                current = - ab(board,depth - 1, - beta , - alpha,book,ply-1,nodes_visited)     
            if current > best:
                best = current
                candidate = thisentry[0].move
            if best > alpha:
                alpha = best
            board.erase()
            
            
        for i in xrange(num_legal):
            if alpha >= beta:
                break                
            if table_move and moves[i] == thisentry[0].move: 
                continue
            counter += 1
            board.update(moves[i])
            if board.check_over():
                current = board.get_score()*(1-2*board.player)
            else:
                current = - ab(board,depth - 1, - beta , - alpha,book,ply-1,nodes_visited)
            if current > best:
                best = current
                candidate = moves[i]
            if best > alpha:
                alpha = best
            board.erase()
            

    size = nodes_visited[0] - size #how many nodes were accessed since this ab function was called
    if best >= original_beta:
        node = <char>'l'
    elif best <= original_alpha:
        node = <char>'u'
    else:
        node = <char>'e'
    push_twobig(board.get_key(),best,candidate,depth,size+1,node)
    return best

cdef double ab_run(Board board,int depth,double alpha,double beta,int iterate,dict book,int ply):
    cdef int d
    cdef int player
    cdef double value
    cdef int nodes_visited = 0
    
    if iterate:
        for d in xrange(depth+1):
            value = ab(board,d,alpha,beta,book,ply,&nodes_visited)
            if value:
                break
    else:
        value = ab(board,depth,alpha,beta,book,ply,&nodes_visited)
    return value

def ab_wrapper(Board board,int depth=0,double alpha=-INF,double beta = INF,int iterate = 0,dict book = {},ply = -1):
    cdef double value
    value = ab_run(board,depth,alpha,beta,iterate,book,ply)
    return value

cdef int64 seed[2]
# seed[1] = 42
seed[1] = random.randint(0,0xffffffffffffffff)

cdef int64 rng():
    return random.getrandbits(64)

# cdef int64 rng():
#     cdef int64 x,y
#     x=seed[0]
#     y=seed[1]
#     seed[0]=y
#     x^=x<<23
#     seed[1]=x^y^(x>>17)^(y>>26)
#     return seed[1]+y

def debug_wrapper():
    count = 0
    i = 0
    cdef char n
    cdef char b
    while i<PRIME and count < 100:
        i +=1
        if twobig[i].big.key != -1 or twobig[i].small.key != -1:
            count += 1
            n = twobig[i].small.node
            b = twobig[i].big.node
            if n == 'n':
                np = 'n'
            elif n == 'x':
                np = 'x'
            elif n == 'u':
                np = 'u'
            elif n == 'l':
                np = 'l'
            elif n == 'e':
                np = 'e'
            else:
                np = 'unknown'
            if b == 'n':
                bp = 'n'
            elif b == 'x':
                bp = 'x'
            elif b == 'u':
                bp = 'u'
            elif b == 'l':
                bp = 'l'
            elif b == 'e':
                bp = 'e'
            else:
                bp = 'unknown'
            print (hex(twobig[i].small.key),twobig[i].small.value,twobig[i].small.depth,hex(twobig[i].small.move),twobig[i].small.size,np)
            print (hex(twobig[i].big.key),twobig[i].big.value,twobig[i].big.depth,hex(twobig[i].big.move),twobig[i].big.size,bp)
    
cpdef int128 key_symm(int128 key):
    cdef int128 new = 0
    for i in xrange(COLUMNS):
        new ^= (COL_MASK & (key >> ((ROWS+1) * i))) << ((ROWS+1)*(COLUMNS-i-1))
    return new

def reset_pulls_pushes():
    global pushes
    global bigpulls
    global newpulls
    pushes = 0
    bigpulls = 0
    newpulls = 0
def pulls_count():
    return newpulls,bigpulls
def pushes_count():
    return pushes
def init_wrapper():
    init_twobig()

    
def print_from_key(key):
    cdef int128 board1 = 0L
    cdef int128 board2 = 0L
    
    for i in xrange(COLUMNS):
        cap = 0#sentinal to find the column cap for the key
        for j in range(ROWS+1)[::-1]:
            if cap:
                if (key >> (i*(1+ROWS)+j))&1:
                    board1 += 1L << (i*(1+ROWS)+j)
                else:
                    board2 += 1L << (i*(1+ROWS)+j)
            else:
                if (key >> (i*(1+ROWS)+j))&1:
                    cap = 1
                    
        
        
    big = ''
    for i in xrange(ROWS):
        s = '|'
        for j in xrange(COLUMNS):
            s += ' '
            if (board1 >> (i + j * (1 + ROWS))) & 1:
                s += 'x'
            elif (board2 >> (i + j * (1 + ROWS))) & 1:
                s += 'o'
            else:
                s += ' '
            s += '|'
        big = s + '\n' + big
    print big
    return board1,board2
    
    
cpdef void smart_debug(Board board):
    cdef int128 moves[MAX_COLUMNS]
    cdef int num_moves
    num_moves = board.get_smart_moves(moves)
    if num_moves == -1:
        print 'win' , log(moves[0])/7
        return
    if num_moves == 0:
        print 'lose', log(moves[0])/7
        return
    print 'number of moves', num_moves
    print [log(moves[i])/7 for i in xrange(num_moves)]
    
cpdef void insert_debug():
    cdef int scores[7]
    cdef move moves[7]
    push_scores = [0,11,-8,3,2,1,-10]
    push_moves = [0,1,2,3,4,5,6]
    for i in xrange(7):
        insert(scores,moves,push_scores[i],push_moves[i],i)
    print scores
    print moves
    

def push_debug():
    init_twobig()
    global pushes
    print pushes
    push_twobig(3,1,7,4,2,<char>'e')
    print twobig[17].big.key != -1
    print twobig[17].big.key != <int64> -1
    
    print pushes
    
def pull_debug():    
    entry = pull_twobig(3)
    
    print entry.value
    print entry.depth
    
def get_twobig_entry(i):
#     print (twobig[i].big.key,twobig[i].big.value,twobig[i].big.depth,twobig[i].big.move,twobig[i].big.size,twobig[i].big.node)
#     print (twobig[i].small.key,twobig[i].small.value,twobig[i].small.depth,twobig[i].small.move,twobig[i].small.size,twobig[i].small.node)
    return twobig[i%PRIME]
    
    

cpdef bitindex_debug(int128 bitmove):    
    print bitindex(bitmove)

In [5]:
rows = 6
columns = 6

board = Board(rows,columns)
import sys
import time
t0 = time.clock()
init_wrapper()
for depth in xrange(rows*columns):
    sys.stdout.write('{}\n'.format(depth))
    sys.stdout.flush()
    time.sleep(0.0001)
    r = ab_wrapper(board,depth)
print
print 'time elapsed: {}'.format(time.clock() - t0)
print r

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35

time elapsed: 101.23986213
-1.0


In [11]:
import time
rows = 6
columns = 7
board = Board(rows,columns)
init_wrapper()
t0 = time.clock()
print ab_wrapper(board,rows*columns,alpha = -1,beta = 1)
print time.clock() - t0
init_wrapper()
t0 = time.clock()
print ab_wrapper(board,rows*columns)
print time.clock() - t0

1.0
693.917061703
2.0
1057.15555503


In [12]:
import time
import sys
rows = 6
columns = 7
for i in range(0,rows*columns+1,2)[::-1]:
    change_symply(i)
    board = Board(rows,columns)
    init_wrapper()
    t0 = time.clock()
    result = ab_wrapper(board,rows*columns)
    t1 = time.clock() - t0
    print "symply = {}, result = {}, time = {}".format(i , result , t1)
    sys.stdout.flush()
    time.sleep(0.001)

symply = 42, result = 2.0, time = 355.199756828
symply = 40, result = 2.0, time = 357.361583159
symply = 38, result = 2.0, time = 353.268871099
symply = 36, result = 2.0, time = 348.564977933
symply = 34, result = 2.0, time = 326.738307653
symply = 32, result = 2.0, time = 324.388638324
symply = 30, result = 2.0, time = 320.221985611
symply = 28, result = 2.0, time = 305.704616029
symply = 26, result = 2.0, time = 289.390411061
symply = 24, result = 2.0, time = 282.196795947
symply = 22, result = 2.0, time = 294.183314163
symply = 20, result = 2.0, time = 299.20408905
symply = 18, result = 2.0, time = 299.595395377
symply = 16, result = 2.0, time = 301.675528503
symply = 14, result = 2.0, time = 310.931458069
symply = 12, result = 2.0, time = 316.143990154
symply = 10, result = 2.0, time = 324.154589235
symply = 8, result = 2.0, time = 336.186185781
symply = 6, result = 2.0, time = 347.203202828
symply = 4, result = 2.0, time = 348.156461154
symply = 2, result = 2.0, time = 426.3405524

In [41]:
0xffffffffffffffff

18446744073709551615L

In [4]:
import time
import sys
opening = {}
PLY = 12
for iterate in xrange(1):
    times = []
    mean_time = 0
    nodes_visited_ar = []
    mean_nodes = 0
    init_wrapper()
    filename = '../c4_benchmark/Test_L2_R2'
    bench = open(filename , 'r')    
    correct = 0
    if iterate == 0:
        print 'no iterative deepening'
    else:
        print
        print 'iterative deepening'
        mean_time = 0
        times=[]
        init_wrapper()
    for i in xrange(1000):

        moves,rawresult = bench.readline().split()
        rawresult = int(rawresult)
        moves = map(int,list(moves))
        moves = [move-1 for move in moves]
        n = len(moves)  
        
        if rawresult * (1-2*(n%2)) >= 0:
            result = rawresult*2
        else:
            result = rawresult * 2 - 1*(1-2*(rawresult<0))
        
        board = Board()
        for move in moves:
            board.p_update(move)
        
#         get_move_debug(board)
    #     print
    #     board.print_board()
        max_depth = 44 - len(moves)
        t0 = time.clock()
        
        ab_result = ab_wrapper(board,depth = max_depth,iterate=iterate,book = opening,ply=PLY)
        
        t1 = time.clock()
        times.append(t1 - t0)
        mean_time += (times[-1] - mean_time) / float(i+1)
        nodes_visited = board.get_nodes_visited()
        nodes_visited_ar.append(nodes_visited)
        mean_nodes += (nodes_visited_ar[-1] - mean_nodes) / float(i+1)
#         print ab_result
        if ab_result == result:
            correct += 1
        else:
            print ' '
            print board.print_board()
            print [move+1 for move in moves]
            print n
            print rawresult
            print result
            print ab_result
        sys.stdout.write('\rsolving problem {:4d}, solved {:3.1f}%, average time {: 3.6f}s, mean nodes visited {: 12,.0f}, knodes/s {:5.0f}'.format(
                i+1 , float(correct) * 100. / (i+1) , mean_time,mean_nodes,mean_nodes/mean_time/1000))
        sys.stdout.flush()

no iterative deepening
solving problem  124, solved 100.0%, average time  0.045798s, mean nodes visited       45,533, knodes/s   994

KeyboardInterrupt: 

In [16]:
import time
import sys
def write_book(book,filename):
    open(filename,'w').close()
    file_ = open(filename,'a')
    for key in book:
        file_.write("{:<20d} {: 1d}\n".format(key,book[key]))
    file_.close()
    
def read_book(book,filename):
    book.clear()
    for line in open(filename,'r'):
        key,result = line.split()
        key = long(key)
        result = int(result)
        book[key] = result
    
    
def ply_search(board , plys_to_search , book , verbose = 1 , save = 1):
    filename = "../c4books/book{}x{}strong.txt".format(*board.get_dimensions())
    global counter
    key = board.get_key_wrapper()
    symmkey = key_symm(key)
    if key in book:
        return
    if plys_to_search == 0:
        if board.is_over():
            result = -1
        else:
            result = int(ab_wrapper(board,reduce(lambda x,y:x*y,board.get_dimensions())))
        book[key] = result
        if save:
            with open(filename,'a') as file_:
                file_.write("{:<20d} {: 1d}\n".format(key,book[key]))
        counter +=1
        if symmkey != key and not symmkey in book:
            book[symmkey] = result
            if save:
                with open(filename,'a') as file_:
                    file_.write("{:<20d} {: 1d}\n".format(symmkey,book[symmkey]))
            counter += 1   
        nodes.append(board.get_nodes_visited())
        times.append(time.clock())        
        start = max(0,len(times)-1000)
        if times[-1] - times[start] > 0.00000001:
            sys.stdout.write('\rwriting move {:8d}'.format(counter) + ', knodes/s = {:>5.0f}  '.format(0.001*
                (nodes[-1]-nodes[start])/(times[-1]-times[start])))
            sys.stdout.flush()
        return
    elif board.is_over():
        return
    
    for move in board.p_get_legal():
        board.p_update(move)
        ply_search(board,plys_to_search - 1,book,verbose-1,save-1)
        if verbose > 0:
            moves = ''.join(map(str,board.get_log()))
            sys.stdout.write('\r' + ' '*60)#whiteout
            sys.stdout.write('\rfinished moves '+moves+ ', time elapsed: {: 3.2f}s\n'.format(time.clock()-t0))
            sys.stdout.flush()
            time.sleep(0.0001)
#         if save > 0:
#             write_book(book,filename)
#             sys.stdout.write('\r' + ' '*40 + '\r')#whiteout
# #             sys.stdout.write('\rsaved to disc\n'.format(time.clock()-t0))
        board.p_erase()

def Solve(rows = 6,columns = 7,book = {},write_ply = 8,verbose = 1,read = True,write = True):
    global times
    global nodes
    global counter
    global t0
    global file_
    t0 = time.clock()
    times = [t0]
    nodes = [0]    
    filename = "../c4books/book{}x{}strong.txt".format(rows,columns)
    board = Board(rows,columns)
    init_wrapper()
    key = board.get_key_wrapper()
    if read:
        try:
            print "reading file"
            read_book(book,filename)

        except:
            print "no file to read"
            pass
    counter = len(book)
    ply_search(board,write_ply, book,verbose,write)    
#     write_book(book,filename)
    
    
   

In [21]:
Solve(5,4,read = False,write = False)

finished moves 2, time elapsed:  0.58s
finished moves 1, time elapsed:  0.84s
finished moves 3, time elapsed:  1.11s
finished moves 0, time elapsed:  1.36s


In [23]:
board = Board(6,6)
init_wrapper()
ab_wrapper(board)

0.0

In [109]:
rows = 4
for columns in xrange(4,9):
    print "solving {}x{}".format(rows,columns)
    Solve(rows,columns,verbose = 2)

solving 4x4
reading file
finished moves 22, time elapsed:  0.27s                     
finished moves 21, time elapsed:  0.29s                     
finished moves 23, time elapsed:  0.31s                     
finished moves 20, time elapsed:  0.32s                     
finished moves 2, time elapsed:  0.33s                      
finished moves 12, time elapsed:  0.34s                     
finished moves 11, time elapsed:  0.36s                     
finished moves 13, time elapsed:  0.38s                     
finished moves 10, time elapsed:  0.39s                     
finished moves 1, time elapsed:  0.40s                      
finished moves 32, time elapsed:  0.41s                     
finished moves 31, time elapsed:  0.43s                     
finished moves 33, time elapsed:  0.45s                     
finished moves 30, time elapsed:  0.47s                     
finished moves 3, time elapsed:  0.47s                      
finished moves 02, time elapsed:  0.48s                     

In [96]:
nodes[-1000:]

[1895485756,
 1895485757,
 1898383055,
 1902220752,
 1902363780,
 1904281402,
 1909488159,
 1911397017,
 1912736691,
 1913953953,
 1916649020,
 1924887713,
 1929241956,
 1929242077,
 1929242078,
 1929242079,
 1929242080,
 1929242081,
 1931610358,
 1931610370,
 1931610371,
 1931610372,
 1931610373,
 1932078358,
 1932078362,
 1932078363,
 1932078364,
 1932078365,
 1932408725,
 1932408755,
 1932408756,
 1932408757,
 1932408758,
 1935075374,
 1935075379,
 1935075380,
 1935075381,
 1936174157,
 1936174162,
 1936174163,
 1936174164,
 1939426381,
 1939426389,
 1939426397,
 1939426405,
 1939426413,
 1939426419,
 1939426420,
 1939426421,
 1939426422,
 1940462956,
 1940462962,
 1940462963,
 1940530561,
 1940530601,
 1940530602,
 1942751013,
 1942751017,
 1942751018,
 1942751019,
 1942751020,
 1947047410,
 1956091476,
 1959202649,
 1963500858,
 1965149841,
 1966955592,
 1981718415,
 1985459836,
 1996055992,
 2000151516,
 2000234380,
 2001834677,
 2004626974,
 2004774156,
 2004913394,
 2005568897,

In [31]:
for i in xrange(100):
    print get_twobig_entry(i)

{'small': {'node': 120, 'move': 32L, 'value': -7.0, 'depth': 8, 'key': 2539136617387L, 'size': 1L}, 'big': {'node': 108, 'move': 536870912L, 'value': 5.0, 'depth': 17, 'key': 103940311820L, 'size': 836L}}
{'small': {'node': 108, 'move': 67108864L, 'value': 4.0, 'depth': 6, 'key': 3702909108695L, 'size': 2L}, 'big': {'node': 108, 'move': 4L, 'value': -0.0, 'depth': 20, 'key': 103596310789L, 'size': 3270L}}
{'small': {'node': 108, 'move': 1073741824L, 'value': -0.0, 'depth': 4, 'key': 3678054034131L, 'size': 4L}, 'big': {'node': 117, 'move': 2048L, 'value': -0.0, 'depth': 10, 'key': 826141478419L, 'size': 157L}}
{'small': {'node': 108, 'move': 8388608L, 'value': 2.0, 'depth': 12, 'key': 4089091267240L, 'size': 15L}, 'big': {'node': 108, 'move': 524288L, 'value': 1.0, 'depth': 15, 'key': 138945416838L, 'size': 177L}}
{'small': {'node': 117, 'move': 67108864L, 'value': 0.0, 'depth': 3, 'key': 1372625117867L, 'size': 6L}, 'big': {'node': 108, 'move': 524288L, 'value': 0.0, 'depth': 21, 'key

In [3]:
%%cython -a
print 'Hello World'

DistutilsPlatformError: Unable to find vcvarsall.bat

In [10]:
import sys
sys.version


'2.7.14 |Anaconda, Inc.| (default, Oct 15 2017, 03:34:40) [MSC v.1500 64 bit (AMD64)]'

In [9]:
board = Board(6,8)
print board.rows , board.columns

6 8
